Setting up your keys

With Costs:
For OpenAI, visit https://openai.com/api/\
For Anthropic, visit https://console.anthropic.com/\
For Google, visit https://ai.google.dev/gemini-api

Free:
Ollama

When you get your API keys, you need to set them as environment variables by adding them to your .env file.

OPENAI_API_KEY=xxxx\
ANTHROPIC_API_KEY=xxxx\
GOOGLE_API_KEY=xxxx

Afterwards, you may need to restart the Jupyter Lab Kernel (the Python process that sits behind this notebook) via the Kernel menu, and then rerun the cells from the top.

In [15]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
import requests
from IPython.display import Markdown, display, update_display

In [2]:
# import for google
# in rare cases, this seems to give an error on some systems, or even crashes the kernel
# If this happens to you, simply ignore this cell - I give an alternative approach for using Gemini later

import google.generativeai

In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

In [4]:
# Connect to OpenAI, Anthropic

openai = OpenAI()

claude = anthropic.Anthropic()

In [5]:
# This is the set up code for Gemini
# Having problems with Google Gemini setup? Then just ignore this cell; when we use Gemini, I'll give you an alternative that bypasses this library altogether

google.generativeai.configure()

In [8]:
# Ollama Connection
OLLAMA_API = "http://localhost:11434/api/chat"
OLLAMA_HEADERS = {"Content-Type": "application/json"}

In [9]:
# Let's make a conversation between GPT-4o-mini, Claude-3-haiku and Gemini 
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"
ollama_model = "llama3.2"
gemini_model='gemini-1.5-flash'

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

claude_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

ollama_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gemini_system = "You are a very funny chatbot. You agree with a polite chat bot \
but always doing with a new question and all the time making joke of it."

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]
ollama_messages = ["Hi"]
gemini_messages = ["Hi Folks"]

In [21]:
def call_gpt(user_content):
    messages = [{"role": "system", "content": gpt_system}]

    messages.append({"role": "user", "content": user_content})

    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [24]:
print(call_gpt(gpt_messages))

BadRequestError: Error code: 400 - {'error': {'message': "Invalid type for 'messages[1].content[0]': expected an object, but got a string instead.", 'type': 'invalid_request_error', 'param': 'messages[1].content[0]', 'code': 'invalid_type'}}

In [18]:
def call_ollama(user_content):
    messages = [{"role": "system", "content": ollama_system}]
    
    messages.append({"role": "user", "content": user_content})
    
    payload = {
        "model": ollama_model,
        "messages": messages,
        "stream": False
    }
    
    response = requests.post(OLLAMA_API, json=payload, headers=OLLAMA_HEADERS)
    return response.json()['message']['content']


In [ ]:
print(call_ollama("Describe some of the business applications of Generative AI"))

In [ ]:
print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Ollama:\n{ollama_messages[0]}\n")
print(f"Gemini:\n{gemini_messages[0]}\n")

print(response.json()['message']['content'])

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)

In [ ]:
# As an alternative way to use Gemini that bypasses Google's python API library,
# Google has recently released new endpoints that means you can use Gemini via the client libraries for OpenAI!

gemini_via_openai_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

response = gemini_via_openai_client.chat.completions.create(
    model=gemini_model,
    messages=prompts
)
print(response.choices[0].message.content)